In [1]:
import os
import tarfile
from six.moves import urllib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as spy
import sklearn as skl

## Reading data

In [2]:
cwd = os.getcwd()
BASE_DIR = os.path.dirname(cwd)
TRAIN_TEST_DIR = os.path.join(BASE_DIR, 'CH02\\datasets\\housing')

In [3]:
TRAIN_PATH = os.path.join(TRAIN_TEST_DIR, 'strat-train-set.csv')
TEST_PATH = os.path.join(TRAIN_TEST_DIR, 'strat-test-set.csv')

In [4]:
strat_train_set = pd.read_csv(TRAIN_PATH)
strat_test_set = pd.read_csv(TEST_PATH)

In [5]:
print(strat_train_set.shape)
strat_train_set.head()

(16512, 10)


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-121.89,37.29,38.0,1568.0,351.0,710.0,339.0,2.7042,286600.0,<1H OCEAN
1,-121.93,37.05,14.0,679.0,108.0,306.0,113.0,6.4214,340600.0,<1H OCEAN
2,-117.20,32.77,31.0,1952.0,471.0,936.0,462.0,2.8621,196900.0,NEAR OCEAN
3,-119.61,36.31,25.0,1847.0,371.0,1460.0,353.0,1.8839,46300.0,INLAND
4,-118.59,34.23,17.0,6592.0,1525.0,4459.0,1463.0,3.0347,254500.0,<1H OCEAN


In [8]:
housing = strat_train_set.drop('median_house_value', 
                                       axis = 1)
housing_labels = strat_train_set['median_house_value'].copy()

## Select and train a model

In [9]:
housing_num = housing.drop('ocean_proximity', axis = 1)

In [10]:
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
         return self # nothing else to do
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room] 
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room = False)
housing_extra_attribs = attr_adder.transform(housing.values)

In [17]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator,TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self,X ,y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [23]:
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer as Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

num_attribs = list(housing_num)
cat_attribs = ['ocean_proximity']

num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    ('imputer', Imputer(strategy="median")),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
])
cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_attribs)),
    ('one_hot_encoder', OneHotEncoder(sparse=False)),
])
full_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("cat_pipeline", cat_pipeline),
])

In [24]:
housing_prepared = full_pipeline.fit_transform(housing)
housing_prepared

array([[-1.15604281,  0.77194962,  0.74333089, ...,  0.        ,
         0.        ,  0.        ],
       [-1.17602483,  0.6596948 , -1.1653172 , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.18684903, -1.34218285,  0.18664186, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.58648943, -0.72478134, -1.56295222, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.78221312, -0.85106801,  0.18664186, ...,  0.        ,
         0.        ,  0.        ],
       [-1.43579109,  0.99645926,  1.85670895, ...,  0.        ,
         1.        ,  0.        ]])

### Training and Evaluating on the Training Set

### Linear Regression Model

In [25]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

LinearRegression()

In [26]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
print("Predictions:\t", lin_reg.predict(some_data_prepared))
print("Labels:\t\t", list(some_labels))

Predictions:	 [210644.60459286 317768.80697211 210956.43331178  59218.98886849
 189747.55849879]
Labels:		 [286600.0, 340600.0, 196900.0, 46300.0, 254500.0]


In [28]:
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

68628.19819848923

It didn't work very well, so let's try another model

### Decision Tree Regressor

In [32]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

DecisionTreeRegressor()

In [33]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.0

It's overfitting, the model makes predictions = labels. Let's try with k-fold cross validation

In [35]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                    scoring="neg_mean_squared_error", cv=10)
rmse_scores = np.sqrt(-scores)

In [36]:
def display_scores(scores):
    print('Scores: ', scores)
    print('Mean: ', scores.mean())
    print('Standar deviation: ', scores.std())
    
display_scores(rmse_scores)

Scores:  [68753.07219578 65539.1853781  68341.12495194 68704.66908799
 72184.3047887  75763.7734249  70000.89338732 69724.36741505
 75393.37772569 69737.98943795]
Mean:  70414.27577934045
Standar deviation:  3023.121216114137


In [37]:
lin_scores = cross_val_score(lin_reg, housing_prepared, 
                            housing_labels, 
                            scoring = 'neg_mean_squared_error', cv = 10)
lin_rmse_scores= np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Scores:  [66782.73843989 66960.118071   70347.95244419 74739.57052552
 68031.13388938 71193.84183426 64969.63056405 68281.61137997
 71552.91566558 67665.10082067]
Mean:  69052.46136345083
Standar deviation:  2731.674001798347


### Random Forest Regressor

In [38]:
from sklearn.ensemble import RandomForestRegressor 

forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)

RandomForestRegressor()

In [39]:
housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

18608.027563263153

In [40]:
forest_scores = cross_val_score(forest_reg, housing_prepared,
                               housing_labels, 
                               scoring = 'neg_mean_squared_error', 
                               cv = 10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

Scores:  [49706.34988799 47608.00288253 49808.88063241 52218.85227779
 49761.99111303 53097.26166541 48860.28469653 48164.36448961
 53265.96573328 49996.30871126]
Mean:  50248.82620898405
Standar deviation:  1874.387909949765


### Support Vector Machine

In [ ]:
from sklearn.svm import SVR

svm_model = SVR() 
svm_model.fit(housing_prepared, housing_labels)

In [ ]:
housing_predictions = svm_model.predict(housing_prepared)
svm_mse = mean_squared_error(housing_labels, housing_predictions)
svm_rmse = np.sqrt(svm_mse)
svm_rmse

In [ ]:
svm_scores = cross_val_score(svm_model, housing_prepared,
                               housing_labels, 
                               scoring = 'neg_mean_squared_error', 
                               cv = 10)

In [ ]:
svm_rsme_scores = np.sqrt(-svm_scores)
display_scores(svm_rsme_scores)

### Tunning the model

At this point we'd have a list of promising models that perform well. Now we must fine-tune them, to minimize the errors. Let's work with the RandomForestRegressor as this performed better.   

In [41]:
from sklearn.model_selection import GridSearchCV

param_grid = [{'n_estimators':[3, 10, 30], 'max_features':[2, 4, 6, 8]},
              {'bootstrap':[False], 'n_estimators':[3, 10],
               'max_features':[2, 3, 4]}]
forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv = 5,
                       scoring='neg_mean_squared_error')
grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             scoring='neg_mean_squared_error')

In [42]:
grid_search.best_params_

{'max_features': 8, 'n_estimators': 30}

In [43]:
grid_search.best_estimator_

RandomForestRegressor(max_features=8, n_estimators=30)

In [46]:
cvres = grid_search.cv_results_
pd.DataFrame(cvres).loc[:, 'params':'mean_test_score'].head(3)

,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score
0,"{'max_features': 2, 'n_estimators': 3}",-3.995358e+09,-3.921683e+09,-4.331267e+09,-3.679434e+09,-4.340011e+09,-4.053551e+09
1,"{'max_features': 2, 'n_estimators': 10}",-2.795184e+09,-3.167410e+09,-3.096309e+09,-3.160449e+09,-3.257002e+09,-3.095271e+09
2,"{'max_features': 2, 'n_estimators': 30}",-2.575054e+09,-2.929297e+09,-2.882201e+09,-2.642946e+09,-2.879175e+09,-2.781735e+09


In [60]:
a = list()
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    a.append([np.sqrt(-mean_score), params])
for value in sorted(a): 
    print(value)

[49839.682921766056, {'max_features': 8, 'n_estimators': 30}]
[50160.76603599632, {'max_features': 6, 'n_estimators': 30}]
[50378.51886610415, {'max_features': 4, 'n_estimators': 30}]
[51788.103490967165, {'bootstrap': False, 'max_features': 4, 'n_estimators': 10}]
[51954.01310565212, {'max_features': 8, 'n_estimators': 10}]
[52619.386356871546, {'max_features': 6, 'n_estimators': 10}]
[52742.15238144581, {'max_features': 2, 'n_estimators': 30}]
[52917.11874433583, {'max_features': 4, 'n_estimators': 10}]
[52987.36041812152, {'bootstrap': False, 'max_features': 3, 'n_estimators': 10}]
[54645.60143797009, {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}]
[55635.15905764858, {'max_features': 2, 'n_estimators': 10}]
[58143.22292381111, {'bootstrap': False, 'max_features': 4, 'n_estimators': 3}]
[58287.407329236725, {'max_features': 8, 'n_estimators': 3}]
[59456.50335341541, {'max_features': 6, 'n_estimators': 3}]
[60615.92406663791, {'max_features': 4, 'n_estimators': 3}]
[6072

In [50]:
feature_importance = grid_search.best_estimator_.feature_importances_
feature_importance

array([6.65542196e-02, 6.10962413e-02, 4.41894637e-02, 1.50975983e-02,
       1.43099554e-02, 1.50146430e-02, 1.41579215e-02, 3.85164376e-01,
       4.87098062e-02, 1.09633277e-01, 5.69620532e-02, 5.83006823e-03,
       1.57623551e-01, 6.08422409e-05, 1.74121996e-03, 3.85476276e-03])

In [66]:
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
cat_one_hot_attribs = housing[cat_attribs].value_counts().index.tolist()
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importance, attributes), reverse = True)

[(0.3851643764289539, 'median_income'),
 (0.15762355146402834, ('INLAND',)),
 (0.10963327671365669, 'pop_per_hhold'),
 (0.06655421959994931, 'longitude'),
 (0.061096241278001936, 'latitude'),
 (0.05696205318336816, 'bedrooms_per_room'),
 (0.0487098062339218, 'rooms_per_hhold'),
 (0.0441894637450688, 'housing_median_age'),
 (0.01509759827827383, 'total_rooms'),
 (0.01501464302355766, 'population'),
 (0.01430995541137656, 'total_bedrooms'),
 (0.014157921451239425, 'households'),
 (0.0058300682308793554, ('<1H OCEAN',)),
 (0.0038547627566583014, ('ISLAND',)),
 (0.0017412199601221953, ('NEAR BAY',)),
 (6.0842240943815895e-05, ('NEAR OCEAN',))]

Well, from now on it's important to remove those features that are not providing powerful information (e.g. some of the cat_data such as 'NEAR_BAY', etc). And maybe clean the data more o adding more features, and so on, in order to get best model. 

### Evaluating on the test set

After tweaking your models for a while, you eventually have a system that performs
sufficiently well. Now is the time to evaluate the final model on the test set.

In [1]:
final_model = grid_search.best_estimator_
final_model

NameError: name 'grid_search' is not defined

In [ ]:
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

In [ ]:
X_test_prepared = full_pipeline.transform(X_test)

In [ ]:
final_predictions = final_model.predict(X_test_prepared)

In [ ]:
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

In [ ]:
y_test

In [ ]:
pd.Series(final_predictions)

Now comes the project prelaunch phase: you need to present your solution (high‐
lighting what you have learned, what worked and what did not, what assumptions
were made, and what your system’s limitations are), document everything, and create
nice presentations with clear visualizations and easy-to-remember statements (e.g.,
“the median income is the number one predictor of housing prices”).


### Launching and maintaining the system